In [95]:
import pandas as pd
import hashlib

eluniversal = pd.read_csv('eluniversal_2020_06_30_articles.csv')

eluniversal['newspaper_uid'] = 'eluniversal'

missing_titles_mask = eluniversal['title'].isna()

missing_titles_mask.value_counts()

missing_titles = (eluniversal[missing_titles_mask]['url']
    .str.extract(r'(?P<missing_titles>[^/]+)$')
    .applymap(lambda title: title.split('-'))
    .applymap(lambda title_word_list: ' '.join(title_word_list))
    )

eluniversal.loc[missing_titles_mask, 'title'] = missing_titles.loc[:, 'missing_titles']


uids = (eluniversal
            .apply(lambda row: hashlib.md5(bytes(row['url'].encode())), axis=1)
            .apply(lambda hash_object: hash_object.hexdigest())
            
        )

eluniversal['uid'] = uids

eluniversal.set_index('uid', inplace=True)

eluniversal = eluniversal.dropna()

# stripped_body = (
#     eluniversal
#         .apply(lambda row: row['body'], axis=1)
#         .apply(lambda body: list(body))
#         .apply(lambda letters: list(map(lambda letter: letter.replace('\n', '').replace('\r', ''), letters)))
# )

eluniversal['body'] = eluniversal['body'].str.strip()


import nltk
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

def tokenize_column(df, column):
    return (
        df
            .dropna()
            .apply(lambda row: nltk.word_tokenize(row[column]), axis=1)
            .apply(lambda tokens: list(filter(lambda token: token.isalpha(), tokens)))
            .apply(lambda tokens: list(map(lambda token: token.lower(), tokens)))
            .apply(lambda word_list: list(filter(lambda word: word not in stop_words, word_list)))
            .apply(lambda valid_word_list: len(valid_word_list))
    )

eluniversal['n_tokens_title']  = tokenize_column(eluniversal, 'body')
eluniversal 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chris\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chris\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,body,title,url,newspaper_uid,n_tokens_title
uid,,,,,
f3943f533318d77a8edfca9704eb009d,"En poco menos de dos semanas, dos atentados ha...",Similitudes en atentados contra juez y jefe de...,https://www.eluniversal.com.mx/opinion/javier-...,eluniversal,311
283eb15a96262f94562b960bcffe2b18,Christian Nodal es una de las estrellas del re...,"Christian Nodal, edad y secretos del coach de ...",http://www.unionyucatan.mx/articulo/2020/06/02...,eluniversal,194
9f400463f8000642d8078994d25cfb90,El gobierno del Estado de México a través de l...,Citas para verificar Edomex 2020: Entérate,http://www.unionedomex.mx/articulo/2020/06/02/...,eluniversal,264
cc623d53a29733f2df8d7e17243abeca,La modelo Emily Ratajkowski presumió su nuevo ...,emily ratajkowski arrasa con braless durante p...,https://www.viveusa.mx/showbiz/emily-ratajkows...,eluniversal,206
1e6f4ed28824897fe2c39e7e15643dfc,La Fiscalía General del Estado de Oaxaca (FGEO...,Vinculan a sujeto que presuntamente atacó sexu...,https://oaxaca.eluniversal.com.mx/estatal/28-0...,eluniversal,142
...,...,...,...,...,...
471059cb3474380155e71c9784bd279a,"El secretario de Seguridad Ciudadana, Omar Gar...","Tras atentado, García Harfuch vuelve al quiróf...",http://www.eluniversal.com.mx/metropoli/cdmx/g...,eluniversal,263
fb1f2c41f58899f7cc3f2814f88c73e7,"Los melones, las papayas y las sandías son fru...",El riesgo de comprar frutas cortadas,http://www.eluniversal.com.mx/menu/por-que-no-...,eluniversal,280
1396f488f7fc18081cfa742d9e91d205,Juchitán de Zaragoza.— Tres días después de qu...,Localizan cuerpo de estadounidense desaparecid...,https://oaxaca.eluniversal.com.mx/municipios/1...,eluniversal,140


In [96]:
eluniversal['title'].value_counts()

Pide fe de bautismo. Mujer trans quiere ser reconocida por la Iglesia              2
Reparten culpas por liberación de madre de "El Marro"                              1
Cómo consigo una de Beca Benito Juárez de primaria y secundaria                    1
Los antivirales que han ayudado a combatir el Covid-19                             1
mata dos jovenes en chicago porque preguntaron por su estatura                     1
                                                                                  ..
Se prevé que pandemia de Covid-19 pueda durar entre 2 y 3 años: López-Gatell       1
video policias arrollan manifestantes con patrulla en detroit                      1
Audiencia Nacional española revisará si procede extradición inmediata de Lozoya    1
Cruz Azul, con plantel parchado, va a la Copa Por México                           1
Te pagan por conocer Japón en tours virtuales                                      1
Name: title, Length: 132, dtype: int64

In [98]:
eluniversal[eluniversal['title'] == 'Pide fe de bautismo. Mujer trans quiere ser reconocida por la Iglesia'].url

uid
91ce6113d057e629378ac28803c0a50c    https://www.eluniversalqueretaro.mx/nuestras-h...
b28df23d65546beae6a5eba62f9602ee    https://www.eluniversalqueretaro.mx/portada/mu...
Name: url, dtype: object

In [101]:
eluniversal.drop_duplicates(subset=['title'], keep='first', inplace=True)
eluniversal['title'].value_counts()

Reparten culpas por liberación de madre de "El Marro"                              1
10 excelentes peliculas que fueron fracasos en taquilla                            1
No tengo hasta el momento síntomas de Covid-19: AMLO                               1
Acepta Emilio Lozoya extradición de España a México                                1
Casos confirmados de Covid-19 al 29 de junio en Jalisco                            1
                                                                                  ..
Se prevé que pandemia de Covid-19 pueda durar entre 2 y 3 años: López-Gatell       1
video policias arrollan manifestantes con patrulla en detroit                      1
Audiencia Nacional española revisará si procede extradición inmediata de Lozoya    1
Cruz Azul, con plantel parchado, va a la Copa Por México                           1
Te pagan por conocer Japón en tours virtuales                                      1
Name: title, Length: 132, dtype: int64